In [10]:
import numpy as np
import joblib
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from sklearn.datasets import load_diabetes, load_iris
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, cross_val_predict,RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression, SGDRegressor,ElasticNet
from sklearn.svm import LinearSVR,SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score, mean_squared_error,mean_absolute_error
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.exceptions import ConvergenceWarning

In [11]:
warnings.filterwarnings("ignore", category=FutureWarning)

In [12]:
df_co2=pd.read_csv("DATA.csv")
df_co2.head()

,Car,Model,Volume,Weight,CO2,Unnamed: 5
0,Toyota,Aygo,1000,790,99,NaN
1,Mitsubishi,Space Star,1200,1160,95,NaN
2,Skoda,Citigo,1000,929,95,NaN
3,Fiat,500,900,865,90,NaN
4,Mini,Cooper,1500,1140,105,NaN


In [13]:
df_co2_dropped = df_co2.drop(["Car", "Model","Unnamed: 5"], axis=1)
df_co2_dropped


,Volume,Weight,CO2
0,1000,790,99
1,1200,1160,95
2,1000,929,95
3,900,865,90
4,1500,1140,105
5,1000,929,105
6,1400,1109,90
7,1500,1365,92
8,1500,1112,98
9,1600,1150,99


In [14]:
X=df_co2_dropped.drop("CO2",axis=1)
y=df_co2_dropped["CO2"]

In [15]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [19]:
scaler=StandardScaler()
X_train_scaler=scaler.fit_transform(X_train)
X_test_scaler=scaler.fit_transform(X_test)

In [20]:
models = {
    'Regresión Lineal': LinearRegression(),
    'Ridge': Ridge(alpha=10.0),
    'Lasso': Lasso(alpha=0.5),
    'ElasticNet': ElasticNet(alpha=1.0, l1_ratio=0.5),  # Ajusta estos hiperparámetros si es necesario
    'Árbol de Decisión': DecisionTreeRegressor(max_depth=5, random_state=42)
}

# Evaluar modelos
for name, model in models.items():
    print(f"Evaluando {name}:")
    
    # Entrenamiento con cross_val_predict, usando los datos escalados
    predictions = cross_val_predict(model, X_train_scaler, y_train, cv=5)
    
    # Cálculo de métricas
    rmse = np.sqrt(mean_squared_error(y_train, predictions))  # Usar y_train en lugar de y
    mae = mean_absolute_error(y_train, predictions)  # Usar y_train en lugar de y
    
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    print("-" * 50)


Evaluando Regresión Lineal:
RMSE: 6.0446
MAE: 5.0324
--------------------------------------------------
Evaluando Ridge:
RMSE: 5.8898
MAE: 4.9947
--------------------------------------------------
Evaluando Lasso:
RMSE: 6.0480
MAE: 5.0801
--------------------------------------------------
Evaluando ElasticNet:
RMSE: 5.9514
MAE: 5.0309
--------------------------------------------------
Evaluando Árbol de Decisión:
RMSE: 7.0072
MAE: 5.8594
--------------------------------------------------


In [25]:
# Definir el espacio de búsqueda (aleatoria)
param_dist = {
    'alpha': np.logspace(-2, 1, 6)  # Búsqueda en un rango de 6 valores para alpha
}

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(Ridge(max_iter=5000, tol=1e-5, random_state=42),
                                   param_distributions=param_dist,  # Espacio de búsqueda definido
                                   n_iter=10,  # Número de combinaciones aleatorias a probar
                                   cv=5,  # Validación cruzada de 5 pliegues
                                   scoring='neg_mean_squared_error',  # Evaluar según el MSE negativo
                                   n_jobs=-1,  # Usar todos los núcleos disponibles
                                   random_state=42)  # Fijar la semilla para reproducibilidad

# Ajustar el modelo a los datos de entrenamiento
random_search.fit(X_train_scaler, y_train)

# Mostrar los mejores parámetros y el mejor rendimiento (RMSE)
print("Mejores parámetros:", random_search.best_params_)
print("Mejor RMSE:", np.sqrt(-random_search.best_score_))



Mejores parámetros: {'alpha': np.float64(10.0)}
Mejor RMSE: 5.994011859117317


c:\Users\PAMELA\anaconda3\envs\machinelearning\Lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [24]:
# Definir el espacio de búsqueda (búsqueda exhaustiva)
param_grid = {
    'alpha': np.logspace(-2, 1, 6)  # Búsqueda en un rango de 6 valores para alpha
}

# Configurar GridSearchCV
grid_search = GridSearchCV(Ridge(max_iter=5000, tol=1e-5, random_state=42),
                           param_grid,  # Espacio de búsqueda definido
                           cv=5,  # Validación cruzada de 5 pliegues
                           scoring='neg_mean_squared_error',  # Evaluar según el MSE negativo
                           n_jobs=-1)  # Usar todos los núcleos disponibles

# Ajustar el modelo a los datos de entrenamiento
grid_search.fit(X_train_scaler, y_train)

# Mostrar los mejores parámetros y el mejor rendimiento (RMSE)
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor RMSE:", np.sqrt(-grid_search.best_score_))


Mejores parámetros: {'alpha': np.float64(10.0)}
Mejor RMSE: 5.994011859117317
